# Training a `Robust' Adapter with AdapterDrop

This notebook extends our quickstart adapter training notebook to illustrate how we can use AdapterDrop
to robustly train an adapter, i.e. adapters that allow us to dynmically dropp layers for faster multi-task inference.
Please have a look at the original adapter training notebook for more details on the setup.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -Uq adapters
!pip install -q datasets

In [3]:
from datasets import load_dataset
from transformers import RobertaTokenizer

dataset = load_dataset("rotten_tomatoes")
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

## Training

In [ ]:
from adapters import RobertaAdapterModel
from transformers import RobertaConfig

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"},
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

# Add a new adapter
model.add_adapter("rotten_tomatoes")
# Add a matching classification head
model.add_classification_head("rotten_tomatoes", num_labels=2)
# Activate the adapter
model.train_adapter("rotten_tomatoes")

To dynamically drop adapter layers during training, we make use of HuggingFace's `TrainerCallback'.

In [ ]:
import numpy as np
from adapters import AdapterTrainer
from transformers import TrainingArguments, EvalPrediction, TrainerCallback

class AdapterDropTrainerCallback(TrainerCallback):
  def on_step_begin(self, args, state, control, **kwargs):
    skip_layers = list(range(np.random.randint(0, 11)))
    kwargs['model'].set_active_adapters("rotten_tomatoes", skip_layers=skip_layers)

  def on_evaluate(self, args, state, control, **kwargs):
    # Deactivate skipping layers during evaluation (otherwise it would use the
    # previous randomly chosen skip_layers and thus yield results not comparable
    # across different epochs)
    kwargs['model'].set_active_adapters("rotten_tomatoes", skip_layers=None)


training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    remove_unused_columns=False
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

trainer.add_callback(AdapterDropTrainerCallback())

We can now train and evaluate our robustly trained adapter!

In [20]:
trainer.train()
trainer.evaluate()

Step,Training Loss
200,0.573800
400,0.362600
600,0.326900
800,0.318700
1000,0.303100
1200,0.293200
1400,0.282300
1600,0.284900


{'epoch': 6.0,
 'eval_acc': 0.8799249530956847,
 'eval_loss': 0.29594820737838745,
 'eval_mem_cpu_alloc_delta': 112111,
 'eval_mem_cpu_peaked_delta': 214624,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 94487040,
 'eval_runtime': 5.0862,
 'eval_samples_per_second': 209.586}